In [215]:
import vtk
import json
import numpy as np
import pandas as pd
import progressbar
import sys

#need python 3.6 or newer, mostly because ints are now 64bit
assert sys.version_info >= (3, 6)

In [163]:
meshname = 'cylinder'
nodes_name = 'nodes'
ext = '.bin'

In [164]:
with open(meshname+'.json','r') as file:
    info = json.load(file)

In [165]:
#read nodes
nodesfilename = meshname+"_"+nodes_name+ext
nodes_df = None
with open(nodesfilename, 'rb') as f:
    header = f.read(255)
    data = np.fromfile(f, dtype=np.dtype([('id', np.uint64), ('x', np.float64), ('y', np.float64), ('z', np.float64)]))
    nodes_df = pd.DataFrame(data, columns=data.dtype.names)
    nodes_df = nodes_df.set_index('id')
    data = None
    
print('Nodes')
nodes_df.head()

Nodes


,x,y,z
id,,,
1,-0.831470,-5.555702e-01,0.000000
2,-0.707107,-7.071068e-01,0.000000
3,-0.706908,-7.073059e-01,0.199966
4,-0.831338,-5.557664e-01,0.200110
5,-1.000000,-1.224647e-16,0.000000


In [167]:
def ReadVolumeCellBIN( filename, ncells, nodemask ):
        
    print('Reading ', filename)
    with open(filename, 'rb') as f:
        header = f.read(255)
        data = np.fromfile(f, dtype=np.uint64 ) #id, nnodes

        
    
    cell_count : int = 0
    data_count : int = 0
    data_pp_ids = np.empty(shape=(ncells,), dtype=np.uint64)
    data_pp_nodes = np.empty(shape=(ncells,), dtype=object)
    print('Processing ncells = ', ncells)
    while cell_count<ncells:
        n :int = int( data[data_count+1] )
        #cell_id : int = int( data[cell_count] )
        #print(cell_count + n)
        #node_ids = data[(cell_count + 2):(cell_count + 2 + n)]
        data_pp_ids[cell_count] = data[data_count]
        data_pp_nodes[cell_count] = data[(data_count + 2):(data_count + 2 + n)] 

        nodemask[ data_pp_nodes[cell_count]  ] = 1

        data_count += 2+n
        cell_count += 1
        
    
    return pd.DataFrame({'nodes':data_pp_nodes}, index=data_pp_ids)


def ReadBoundaryCellBIN( filename, ncells ):
        
    print('Reading ', filename)
    with open(filename, 'rb') as f:
        header = f.read(255)
        data = np.fromfile(f, dtype=np.uint64 ) #id, nnodes

    cell_count : int = 0
    data_count : int = 0
    data_pp_ids = np.empty(shape=(ncells,), dtype=np.uint64)
    data_pp_volume_ids = np.empty(shape=(ncells,), dtype=np.uint64)
    data_pp_nodes = np.empty(shape=(ncells,), dtype=object)
    print('Processing ncells = ', ncells)
    while cell_count<ncells:
        data_pp_ids[cell_count] = data[data_count]
        data_pp_volume_ids [cell_count] = data[data_count+1] #reference to the volume mesh cell
        n :int = int( data[data_count+2] )

        data_pp_nodes[cell_count] = data[(data_count + 3):(data_count + 3 + n)] 
        data_count += 3+n
        cell_count += 1
    
    return pd.DataFrame({'nodes':data_pp_nodes, 'volumeCellId':data_pp_volume_ids}, index=data_pp_ids)

In [168]:
#read solid cells
cells_df = pd.DataFrame()

#mask to see which nodes are used
#all ids start with 1, ignore the nodemask[0]
nodemask = np.zeros( int(nodes_df.index.max())+1, dtype=np.int8 )

material_names = []

for solid_idx, solid_info in enumerate( info['Solids'] ):
    filename = meshname + '_' + solid_info['Name'] + ext
    ncells = solid_info['NCells']
    
    df = ReadVolumeCellBIN( filename, ncells, nodemask )
    
    df['solidID'] = solid_idx+1
    cells_df = cells_df.append(df)
    material_names.append( {'Name':solid_info['Name'], 'Id':solid_idx+1 } )

#free memory
df = None


print(material_names)

#create new id for solid cells
cells_df['newCellId'] = np.arange(1,cells_df.shape[0]+1,dtype=np.uint64)
print( cells_df.head() )

#leave only used nodes and create new ids
nodes_df = nodes_df.loc[ np.where(nodemask>0)[0] ]
nodemask = None
nodes_df['newPointId'] = np.arange(1,nodes_df.shape[0]+1,dtype=np.uint64)

print( nodes_df.head() )

Reading  cylinder_layers.bin
Processing ncells =  1600
Reading  cylinder_interior.bin
Processing ncells =  13087
[{'Name': 'layers', 'Id': 1}, {'Name': 'interior', 'Id': 2}]
                                                 nodes  solidID  newCellId
1241      [369, 346, 363, 386, 1391, 1368, 1385, 1408]        1          1
1242  [1391, 1368, 1385, 1408, 2223, 2200, 2217, 2240]        1          2
1243      [385, 368, 355, 372, 1407, 1390, 1377, 1394]        1          3
1244  [1407, 1390, 1377, 1394, 2239, 2222, 2209, 2226]        1          4
1245      [384, 371, 354, 367, 1406, 1393, 1376, 1389]        1          5
           x             y         z  newPointId
id                                              
1  -0.831470 -5.555702e-01  0.000000           1
2  -0.707107 -7.071068e-01  0.000000           2
3  -0.706908 -7.073059e-01  0.199966           3
4  -0.831338 -5.557664e-01  0.200110           4
5  -1.000000 -1.224647e-16  0.000000           5


In [169]:
#read boundaries

bound_df = pd.DataFrame()

boundary_names = []

for boundary_idx, boundary_info in enumerate( info['Boundaries'] ):
    filename = meshname + '_' + boundary_info['Name'] + ext
    ncells = boundary_info['NCells']
    
    df = ReadBoundaryCellBIN( filename, ncells )
    df['boundaryID'] = boundary_idx+1
    bound_df = bound_df.append(df)

    boundary_names.append( {'Name':boundary_info['Name'], 'Id':boundary_idx+1 } )
    

    
#free memory
df = None

print(boundary_names)

bound_df['newFaceId'] = np.arange(1,bound_df.shape[0]+1,dtype=np.uint64)
bound_df.head()


Reading  cylinder_ext_surf.bin
Processing ncells =  800
Reading  cylinder_inlet.bin
Processing ncells =  276
Reading  cylinder_outlet.bin
Processing ncells =  278
[{'Name': 'ext_surf', 'Id': 1}, {'Name': 'inlet', 'Id': 2}, {'Name': 'outlet', 'Id': 3}]


,nodes,volumeCellId,boundaryID,newFaceId
491,"[527, 528, 23, 5]",2041,1,1
492,"[23, 528, 529, 35]",2043,1,2
493,"[530, 531, 528, 527]",2045,1,3
494,"[35, 529, 532, 45]",2047,1,4
495,"[531, 533, 529, 528]",2049,1,5


In [188]:
#renumber the nodes in the cells
cells_df['nodes'] = cells_df['nodes'].apply( lambda x: [ nodes_df.loc[i,'newPointId'] for i in x] )
bound_df['nodes'] = bound_df['nodes'].apply( lambda x: [ nodes_df.loc[i,'newPointId'] for i in x] )

In [193]:
#renumber the volume_cell_ids in the boundary cells
bound_df['volumeCellId'] = bound_df['volumeCellId'].apply( lambda x: cells_df.loc[x,'newCellId'] ) 
bound_df.head()10

,nodes,volumeCellId,boundaryID,newFaceId
491,"[527, 528, 23, 5]",801,1,1
492,"[23, 528, 529, 35]",803,1,2
493,"[530, 531, 528, 527]",805,1,3
494,"[35, 529, 532, 45]",807,1,4
495,"[531, 533, 529, 528]",809,1,5


In [229]:

# ====================================
#
#          save points
#
# ====================================
nodes_df.sort_values(by='newPointId', ascending=True, inplace=True)

pts = vtk.vtkPoints()
print('Extracting points', flush=True)
with progressbar.ProgressBar(max_value=nodes_df.shape[0]) as bar:
    for idx, row in enumerate(nodes_df.itertuples(index=False, name='Pandas')):
        pts.InsertNextPoint( getattr(row, "x"), getattr(row, "y"), getattr(row, "z"))
    
# ====================================
#
#          save volume cells
#
# ====================================
cells_df.sort_values(by='newCellId', ascending=True, inplace=True)

#number of nodes : type
vtkcelltypes = {4:vtk.VTK_TETRA, 5:vtk.VTK_PYRAMID, 6:vtk.VTK_WEDGE, 8:vtk.VTK_HEXAHEDRON}
#vtkfacetypes = {3:vtk.VTK_TRIANGLE, 4:vtk.VTK_QUAD}



cells = vtk.vtkCellArray()
celltypes = np.zeros(cells_df.shape[0], dtype=np.uint64)

material_id = vtk.vtkShortArray()
material_id.SetName('Material')
material_id.SetNumberOfComponents(1)
material_id.SetNumberOfTuples(cells_df.shape[0])

print('Extracting volume cells', flush=True)
with progressbar.ProgressBar(max_value=cells_df.shape[0]) as bar:
    for idx, row in enumerate(cells_df.itertuples(index=False, name='Pandas')):
        nodes = getattr(row, "nodes")
        cells.InsertNextCell( len(nodes) )
        for node in nodes:
            cells.InsertCellPoint( int(node)-1 )

        celltypes[idx] = vtkcelltypes[len(nodes)]
        material_id.SetTuple1(idx,  int(getattr(row, "solidID")) )
        bar.update(idx)

ug = vtk.vtkUnstructuredGrid()
ug.SetPoints(pts)
ug.SetCells(celltypes, vol_cells)
ug.GetCellData().AddArray(material_id)

print('Writing vtk unstructured grid')
wr = vtk.vtkDataSetWriter()
wr.SetFileName('ug.vtk')
wr.SetInputData(ug)
wr.Write()

# ====================================
#
#        process the boundary
#
# ================================
bound_df.sort_values(by='newFaceId', ascending=True, inplace=True)

boundary_id = vtk.vtkShortArray()
boundary_id.SetName('BoundaryId')
boundary_id.SetNumberOfComponents(1)
boundary_id.SetNumberOfTuples(bound_df.shape[0])

vol_cell_id = vtk.vtkShortArray()
vol_cell_id.SetName('VolCellId')
vol_cell_id.SetNumberOfComponents(1)
vol_cell_id.SetNumberOfTuples(bound_df.shape[0])

cells = vtk.vtkCellArray()
#celltypes = np.zeros(bound_df.shape[0], dtype=np.uint64)

print('Extracting boundary faces', flush=True)
with progressbar.ProgressBar(max_value=bound_df.shape[0]) as bar:
    for idx, row in enumerate(bound_df.itertuples(index=False, name='Pandas')):
        nodes = getattr(row, "nodes")
        cells.InsertNextCell( len(nodes) )
        for node in nodes:
            cells.InsertCellPoint( int(node)-1 )

#        celltypes[idx] = vtkfacetypes[len(nodes)]
        boundary_id.SetTuple1(idx,  int(getattr(row, "boundaryID")) )
        vol_cell_id.SetTuple1(idx,  int(getattr(row, "volumeCellId"))-1 ) #correct for VTK numbering from 0
        bar.update(idx)

pd = vtk.vtkPolyData()
pd.SetPoints(pts)
pd.SetPolys(cells)
pd.GetCellData().AddArray(boundary_id)
pd.GetCellData().AddArray(vol_cell_id)

print('Writing boundary vtk')
wr = vtk.vtkPolyDataWriter()
wr.SetFileName('boundary.vtk')
wr.SetInputData(pd)
wr.Write()

Extracting points


100% (4523 of 4523) |####################| Elapsed Time: 0:00:00 ETA:  00:00:00


Extracting volume cells


100% (14687 of 14687) |##################| Elapsed Time: 0:00:00 Time:  0:00:00


Writing vtk unstructured grid
Extracting boundary faces


100% (1354 of 1354) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


Writing boundary vtk


1

In [206]:
nodes_df.iloc[0,0:3]

x   -0.83147
y   -0.55557
z    0.00000
Name: 1, dtype: float64